In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from hist import Hist
from lmfit.models import SkewedGaussianModel
from scipy.stats import skewnorm
from pathlib import Path

This notebook is part of the analysis of the cosmogenic background analysis at different underground sites. 
The purpose is to process the distribution of xenon-137 atoms generated from the toy MC code that includes the efficiencies for muon tagging and the n-capture tagging. The processing here is to transform the distributions in a way that can be used by the sensitivity code 

# Distributions

These are the input data from the toy-MC in form of histogram. Bins are in units of counts in 10 years in the full LXe, so we normalize in units of kg/y

In [ ]:
dist_dict = {}

In [ ]:
counts = [6, 7, 27, 35, 85, 96, 119, 136, 116, 97, 84, 55, 48, 41, 18,  8, 13,  1,  6,  2]
bins = [2533., 2566.3, 2599.6, 2632.9, 2666.2, 2699.5, 2732.8, 2766.1, 2799.4, 2832.7, 2866., 2899.3, 2932.6, 2965.9, 2999.2, 3032.5, 3065.8, 3099.1, 3132.4, 3165.7, 3199.]
hints = {'amplitude': 1, 'center':1, 'sigma':0.4, 'gamma':1}
dist_dict['LNGS_Eff70'] = {'counts':counts, 'bins':np.array(bins)/10./4896., 'hints':hints}

In [ ]:
counts = [3, 0, 2, 12, 8, 14, 20, 10, 12, 7, 1, 5, 2, 2, 1, 0, 0, 0, 0, 1]
bins = [1426., 1446.95, 1467.9, 1488.85, 1509.8, 1530.75, 1551.7, 1572.65, 1593.6, 1614.55, 1635.5, 1656.45, 1677.4, 1698.35, 1719.3, 1740.25, 1761.2, 1782.15, 1803.1, 1824.05, 1845.]
hints = {'amplitude': 1, 'center':1, 'sigma':0.4, 'gamma':1}
dist_dict['LNGS_Eff80'] = {'counts':counts, 'bins':np.array(bins)/10./4896., 'hints':hints}

In [ ]:
counts = [1, 1, 2, 1, 5, 10, 6, 12, 13, 15, 14, 5, 4, 6, 3, 1, 0, 0, 0, 1]
bins = [287., 293.7, 300.4, 307.1, 313.8, 320.5, 327.2, 333.9, 340.6, 347.3, 354., 360.7, 367.4, 374.1, 380.8, 387.5, 394.2, 400.9, 407.6, 414.3, 421., ]
hints = {'amplitude': 1, 'center':1, 'sigma':0.4, 'gamma':1}
dist_dict['SURF_Eff70'] = {'counts':counts, 'bins':np.array(bins)/10./4896., 'hints':hints}

In [ ]:
counts = [2, 1, 1, 3, 7, 6, 8, 12, 8, 7, 6, 8, 6, 7, 8, 5, 2, 2, 0, 1]
bins = [193., 197.15, 201.3, 205.45, 209.6, 213.75, 217.9, 222.05, 226.2, 230.35, 234.5, 238.65, 242.8, 246.95, 251.1, 255.25, 259.4, 263.55, 267.7, 271.85, 276., ]
hints = {'amplitude': 1, 'center':1, 'sigma':0.4, 'gamma':1}
dist_dict['SURF_Eff80'] = {'counts':counts, 'bins':np.array(bins)/10./4896., 'hints':hints}

In [ ]:
counts = [129, 1267, 3508, 3233, 1353, 373, 62, 27, 23, 9, 10, 0, 0, 0, 0, 0, 0, 1, 3, 2]
bins = [9., 15.2, 21.4, 27.6, 33.8, 40., 46.2, 52.4, 58.6, 64.8, 71., 77.2, 83.4, 89.6, 95.8, 102., 108.2, 114.4, 120.6, 126.8, 133.]
hints = {'amplitude': 1, 'center':1, 'sigma':0.4, 'gamma':1}
dist_dict['SNOLAB_Eff70'] = {'counts':counts, 'bins':np.array(bins)/10./4896., 'hints':hints}

In [ ]:
counts = [1, 2, 2, 8, 9, 17, 18, 5, 15, 5, 6, 7, 2, 2, 0, 0, 0, 0, 0, 1]
bins = [7., 8.6, 10.2, 11.8, 13.4, 15., 16.6, 18.2, 19.8, 21.4, 23., 24.6, 26.2, 27.8, 29.4, 31., 32.6, 34.2, 35.8, 37.4, 39., ]
hints = {'amplitude': 1, 'center':1, 'sigma':0.4, 'gamma':1}
dist_dict['SNOLAB_Eff80'] = {'counts':counts, 'bins':np.array(bins)/10./4896., 'hints':hints}

In [ ]:
results = {}
for label, v in dist_dict.items():
    print(f'## {label}')
    h = Hist.new.Regular(len(v['counts']), v['bins'][0], v['bins'][-1], name='cts/kg/y').Double()
    h[:] = v['counts']
    model = SkewedGaussianModel() 
    params = model.make_params(amplitude=1, center=1, sigma=0.4, gamma=1)
    result = model.fit(v['counts'], params, x=h.axes[0].centers)
    results[label] = result
    print("Xe137DistributionModel: ")
    print("    dist: 'skewnorm'")
    print(f"    a: {result.best_values['gamma']}")    
    print(f"    loc: {result.best_values['center']}")
    print(f"    scale: {result.best_values['sigma']}\n\n")
    # print(result.fit_report())
    # result.plot_fit(numpoints=100)
    plt.show()

In [ ]:
ref_spec_activ = 2.684796228e-8 # mBq/kg, from R-088.1 materials db
with open(Path('results') / 'spec_activities.txt', 'a') as f:
    for label, r in results.items():
        rv = skewnorm(r.best_values['gamma'],
                    loc = r.best_values['center'],
                    scale = r.best_values['sigma'])
        mean_xe137_spec_activ = rv.mean() * 1000 / (365 * 24 * 3600)
        print(f'{label}\t{mean_xe137_spec_activ}\t{mean_xe137_spec_activ/ref_spec_activ}')
        print(f'{label}\t{mean_xe137_spec_activ}\t{mean_xe137_spec_activ/ref_spec_activ}', file=f)

Let's normalize and scale the units

In [ ]:
pdf = np.array(counts) / float(sum(counts))
bins = np.array(bins)/10./4896.

In [ ]:
h = Hist.new.Regular(20, bins[0], bins[-1], name='cts/kg/y').Double()
h[:] = counts
h.plot()
plt.show()

The output of the next cell can be copied in the sensitivity configuration file for use as a discrete distribution from which samples are drawn

In [ ]:
bin_centers = h.axes[0].centers
print("counts: [", end='')
print(*counts, sep=', ', end=']\n')
print("bin_centers: [", end='')
print(*bin_centers, sep=', ', end=']\n')

Now we fit with a function so we can use a continuous distribution from sampling. We use `lmfit` because it's so much more convenient

In [ ]:
from lmfit.models import SkewedGaussianModel
model = SkewedGaussianModel() 
params = model.make_params(amplitude=1, center=1, sigma=0.4, gamma=1)
result = model.fit(counts, params, x=bin_centers)
print(result.fit_report())
result.plot_fit(numpoints=100)
h.plot()
# plt.semilogy()

We can now apply the fitting results to the scipy equivalent distribution, which has a fast implementation. 

In [ ]:
from scipy.stats import skewnorm
x = np.linspace(bins[0], bins[-1], 100)
h.plot()
plt.plot(x, result.params['amplitude'].value*skewnorm.pdf(x, result.params['gamma'].value, 
                                                          loc=result.params['center'].value, 
                                                          scale=result.params['sigma'].value),
                        'r-', lw=2, alpha=0.6, label='skewnorm pdf')


## Test Sampling

Here we show how one would reload and use the saved fitted function

In [ ]:
rv = skewnorm(result.params['gamma'].value,loc=result.params['center'].value, 
                                            scale=result.params['sigma'].value)
samples = rv.rvs(size=1000)
h_sampled= Hist.new.Regular(20, min(samples), max(samples), name='E').Double()
h_sampled.fill(samples)
h_sampled.plot()
plt.plot(x, result.params['amplitude'].value*rv.pdf(x),
                        'r-', lw=2, alpha=0.6, label='skewnorm pdf')
plt.show()

In [ ]:
rv.mean()

Here is how one could do the sampling from the discrete distribution

In [ ]:
x = np.random.choice(len(pdf), size=1000, p=pdf)
resampled = np.zeros(len(counts))
for k in x:
    resampled[k] += 1
h_resampled = Hist.new.Regular(20, bins[0], bins[-1], name='cts/kg/y').Double()
h_resampled[:] = resampled
h_resampled.plot()
plt.show()

# SNOLAB Xe137 Distribution

In [ ]:
pdf = np.array(counts) / float(sum(counts))
bins = np.array(bins)/10./4896.

In [ ]:
import matplotlib.pyplot as plt
from hist import Hist
h = Hist.new.Regular(20, bins[0], bins[-1], name='E').Double()
h[:] = counts
h.plot()
plt.show()

This can be copied in the sensitivity configuration file

In [ ]:
bin_centers = h.axes[0].centers
print("counts: [", end='')
print(*counts, sep=', ', end=']\n')
print("bin_centers: [", end='')
print(*bin_centers, sep=', ', end=']\n')

In [ ]:
from lmfit.models import Model, LognormalModel, ExponentialGaussianModel, LinearModel, \
                         ConstantModel, GaussianModel, SkewedVoigtModel, SkewedGaussianModel
model = SkewedGaussianModel()
params = model.make_params(amplitude = 2000, center = -0.5, sigma=2, gamma=1)
result = model.fit(counts, params, x=bin_centers)
print(result.fit_report())
result.plot_fit(numpoints=100)
h.plot()
plt.semilogy()

In [ ]:
from scipy.stats import skewnorm
x = np.linspace(bins[0], bins[-1], 100)
h.plot()
plt.plot(x, result.params['amplitude'].value*skewnorm.pdf(x, result.params['gamma'].value, 
                                                          loc=result.params['center'].value, 
                                                          scale=result.params['sigma'].value),
                        'r-', lw=2, alpha=0.6, label='skewnorm pdf')
